# Advent of Code 2020 Day 5: Binary Boarding

## Import libraries

In [1]:
import numpy as np

## Part 1

### Description Part 1

You board your plane only to discover a new problem: you dropped your boarding pass! You aren't sure which seat is yours, and all of the flight attendants are busy with the flood of people that suddenly made it through passport control.

You write a quick program to use your phone's camera to scan all of the nearby boarding passes (your puzzle input); perhaps you can find your seat through process of elimination.

Instead of zones or groups, this airline uses binary space partitioning to seat people. A seat might be specified like FBFBBFFRLR, where F means "front", B means "back", L means "left", and R means "right".

The first 7 characters will either be F or B; these specify exactly one of the 128 rows on the plane (numbered 0 through 127). Each letter tells you which half of a region the given seat is in. Start with the whole list of rows; the first letter indicates whether the seat is in the front (0 through 63) or the back (64 through 127). The next letter indicates which half of that region the seat is in, and so on until you're left with exactly one row.

For example, consider just the first seven characters of FBFBBFFRLR:

Start by considering the whole range, rows 0 through 127.
F means to take the lower half, keeping rows 0 through 63.
B means to take the upper half, keeping rows 32 through 63.
F means to take the lower half, keeping rows 32 through 47.
B means to take the upper half, keeping rows 40 through 47.
B keeps rows 44 through 47.
F keeps rows 44 through 45.
The final F keeps the lower of the two, row 44.
The last three characters will be either L or R; these specify exactly one of the 8 columns of seats on the plane (numbered 0 through 7). The same process as above proceeds again, this time with only three steps. L means to keep the lower half, while R means to keep the upper half.

For example, consider just the last 3 characters of FBFBBFFRLR:

Start by considering the whole range, columns 0 through 7.
R means to take the upper half, keeping columns 4 through 7.
L means to take the lower half, keeping columns 4 through 5.
The final R keeps the upper of the two, column 5.
So, decoding FBFBBFFRLR reveals that it is the seat at row 44, column 5.

Every seat also has a unique seat ID: multiply the row by 8, then add the column. In this example, the seat has ID 44 * 8 + 5 = 357.

Here are some other boarding passes:

BFFFBBFRRR: row 70, column 7, seat ID 567.
FFFBBBFRRR: row 14, column 7, seat ID 119.
BBFFBBFRLL: row 102, column 4, seat ID 820.
As a sanity check, look through your list of boarding passes. What is the highest seat ID on a boarding pass?

### Import data

In [2]:
with open('day_5_input.txt') as input:
    bplist = [line.strip() for line in input.read().split('\n') if line]

In [3]:
bplist[:10]

['FFBBFFFLRL',
 'FFBBFBBRRL',
 'FBBBFFBLRL',
 'BBFBFFBLRR',
 'BFBBBFFLLL',
 'BFBBBBBLLR',
 'FBFBFBFLLR',
 'BFBFBBFLLR',
 'FBBFBFBLLL',
 'BBBFBFBRLL']

In [4]:
print(len(bplist))

932


In [5]:
# Create a test list and dictionary with answers to check answers
bptestdic = [{'BFFFBBFRRR': 'row 70, column 7, seat ID 567', 
              'FFFBBBFRRR': 'row 14, column 7, seat ID 119', 
              'BBFFBBFRLL': 'row 102, column 4, seat ID 820'}]

bptest = ['BFFFBBFRRR', 'FFFBBBFRRR', 'BBFFBBFRLL']

print(bptest, bptestdic)

['BFFFBBFRRR', 'FFFBBBFRRR', 'BBFFBBFRLL'] [{'BFFFBBFRRR': 'row 70, column 7, seat ID 567', 'FFFBBBFRRR': 'row 14, column 7, seat ID 119', 'BBFFBBFRLL': 'row 102, column 4, seat ID 820'}]


### Use binary sort to find row number

In [6]:
# Sort values
bptest.sort()
bplist.sort()

In [7]:
# Write binary sort & assign values
# 'F' means lower, 'B' means upper

def binary_sort(input):

    input_index = 0
    pass_list_rows = []
    pass_list_cols = []
    
    # Take entry in list
    while input_index < len(input):
            # Parse characters in entry, keep only Bs & Fs
            rowcol = input[input_index]
            rows = "".join([char for char in rowcol if char in ('B', 'F')])
            cols = "".join([char for char in rowcol if char in ('L', 'R')])
            # Reset character index
            char_index = 0
            # Reset row bounds
            lower_row = 0
            upper_row = 2 ** 7 - 1
            while char_index < len(rows):
                char_row = rows[char_index]
                row_bounds = np.array([lower_row, upper_row])
                # If character is 'F', reset upper bound to int(median(bounds)) (median without decimal to round down)
                if char_row == 'F':
                    upper_row = int(np.median(row_bounds))
                # If character is 'B', reset lower bound to int(median(bounds)) + 1 (median without decimal + 1 to round up)
                elif char_row == 'B':
                    lower_row = int(np.median(row_bounds)) + 1
                # Continue to next character until no more characters
                char_index += 1
            # Add either bound to a list of the row numbers for each line
            pass_list_rows += [upper_row]
            # Reset character index
            char_index = 0
            # Reset col bounds
            lower_col = 0
            upper_col = 2 ** 3 - 1
            while char_index < len(cols):
                char_col = cols[char_index]
                col_bounds = np.array([lower_col, upper_col])
                # If character is 'L', reset lower bound to int(median(bounds)) + 1 (median without decimal + 1 to round up)
                if char_col == 'L':
                    upper_col = int(np.median(col_bounds))
                    # If character is 'R', reset lower bound to int(median(bounds)) + 1 (median without decimal + 1 to round up)
                elif char_col == 'R':
                    lower_col = int(np.median(col_bounds)) + 1
                char_index += 1
            # Add either bound to a list of the col numbers for each line
            pass_list_cols += [upper_col]
            # Go to the next line
            input_index += 1
            # Calculate seat IDs
            seat_id = np.array(pass_list_rows) * 8 + np.array(pass_list_cols)
        
    return seat_id

In [8]:
# Test function with test list
seat_id_test = binary_sort(bptest)

# Get max seat ID
print(max(seat_id_test))

820


In [9]:
# Check answer
bptestdic

[{'BFFFBBFRRR': 'row 70, column 7, seat ID 567',
  'FFFBBBFRRR': 'row 14, column 7, seat ID 119',
  'BBFFBBFRLL': 'row 102, column 4, seat ID 820'}]

In [10]:
# Run function on input list
seat_id_list = binary_sort(bplist)

# Get max seat ID
print(max(seat_id_list))

947


## Solution: The highest seat ID on a boarding pass is 947

## Part 2

### Description Part 2

Ding! The "fasten seat belt" signs have turned on. Time to find your seat.

It's a completely full flight, so your seat should be the only missing boarding pass in your list. However, there's a catch: some of the seats at the very front and back of the plane don't exist on this aircraft, so they'll be missing from your list as well.

Your seat wasn't at the very front or back, though; the seats with IDs +1 and -1 from yours will be in your list.

What is the ID of your seat?

In [11]:
seat_id_list.sort()

In [12]:
# Write fn to check for nonsequential

def nonseq(input):

    # Set index to 0 to start
    item_index = 0
    # Create an empty list to store missing IDs
    missing_seat = []
    
    while item_index < len(input) - 1:
        i = input[item_index]
        # Get the sequential value for previous and next in list
        next_expect = i + 1
        next_actual = input[item_index + 1]
        # Check if the values are actually sequential or not
        if next_actual != next_expect:
            missing_seat += [next_expect]
        # Go to next item in list
        item_index += 1
            
    return missing_seat

In [13]:
# Run function on input list
seat_id_missing = nonseq(seat_id_list)

print(seat_id_missing)

[636]


## Solution: The ID of my seat (missing from the list) is 636 